# Importando Bibliotecas

In [5]:
import glob

import pandas as pd
import numpy as np

from scipy.io import wavfile

from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import ShortTermFeatures

# Funções

In [3]:
def specprop(filename):
    '''
    Function to get the acoustic and speech properties of the voice
    '''
    
    fs, data = wavfile.read(filename)

    if data.ndim > 1:
        data = data[:, 0]

    spec = np.abs(np.fft.rfft(data))
    freq = np.fft.rfftfreq(len(data), d=1/fs)

    assert len(spec) == len(freq)

    amp = spec / spec.sum()
    amp_cumsum = amp.cumsum()

    assert len(amp_cumsum) == len(freq)

    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5])]
    mode = freq[amp.argmax()]
    q25 = freq[len(amp_cumsum[amp_cumsum < 0.25])]
    q75 = freq[len(amp_cumsum[amp_cumsum < 0.75])]
    z = amp - amp.mean()
    w = amp.std()
    iqr = q75 - q25
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4
    
    F, f_names = ShortTermFeatures.feature_extraction(data, fs, 0.050*fs, 0.025*fs)
    props = [
        {'name': 'mean', 'value': mean},
        {'name': 'sd', 'value': sd},
        {'name': 'median', 'value': median},
        {'name': 'mode', 'value': mode},
        {'name': 'q25', 'value': q25}, 
        {'name': 'q75', 'value': q75},
        {'name': 'iqr', 'value': iqr},
        {'name': 'skew', 'value': skew},
        {'name': 'kurt', 'value': kurt}
        ]
    for i in range(8):
        props.append({
            'name': f_names[i],
            'value': F[i].mean()
        })
        
    props = pd.DataFrame(props)
    props = props.T
    props.reset_index(inplace=True, drop=True)
    props.columns = ['mean', 'sd', 'median', 'mode', 'q25', 'q75', 'iqr', 'skew', 'kurt', 'zcr', 'energy', 'energy_entropy', 'spectral_centroid', 'spectral_spread', 'spectral_entropy', 'spectral_flux', 'spectral_rolloff']
    props.drop(0, axis=0, inplace=True)
    
    return props

# Lendo e transformando os arquivos

In [3]:
male_file = glob.glob(f'..\\dados\\male\\*.wav')
female_file = glob.glob(f'..\\dados\\female\\*.wav')

In [4]:
df_male = pd.DataFrame()
for file in male_file:
    df_temp_m = specprop(file)
    df_male = pd.concat([df_male, df_temp_m])

In [5]:
df_female = pd.DataFrame()
for file in female_file:
    df_temp_f = specprop(file)
    df_female = pd.concat([df_female, df_temp_f])

In [6]:
df_male.head()

,mean,sd,median,mode,q25,q75,iqr,skew,kurt,zcr,energy,energy_entropy,spectral_centroid,spectral_spread,spectral_entropy,spectral_flux,spectral_rolloff
1,1808.092287,1990.436004,782.362729,98.835275,365.723794,3184.69218,2818.968386,5.096127,36.714087,0.132613,0.014204,2.776143,0.22061,0.207588,0.813328,0.007252,0.211261
1,2200.046049,2274.187035,955.15929,117.462955,351.461525,4268.851065,3917.38954,7.525938,85.611556,0.151185,0.031811,2.755876,0.257105,0.227426,0.944803,0.008513,0.247871
1,1675.355751,2147.506127,556.424261,119.998089,225.474117,2382.637219,2157.163102,8.714798,115.775205,0.112272,0.014426,2.627035,0.228569,0.225899,0.689215,0.015155,0.216506
1,1974.211667,1986.024539,925.484438,144.355789,451.305037,3254.032959,2802.727923,4.360345,26.907695,0.182504,0.015273,2.742456,0.267155,0.213,1.016115,0.008579,0.297207
1,1798.853498,1842.831468,949.25,135.25,376.75,2776.0,2399.25,6.768834,69.747551,0.089617,0.016476,2.795043,0.201541,0.208583,0.721911,0.012701,0.155377


In [7]:
df_female.head()

,mean,sd,median,mode,q25,q75,iqr,skew,kurt,zcr,energy,energy_entropy,spectral_centroid,spectral_spread,spectral_entropy,spectral_flux,spectral_rolloff
1,1597.568747,1864.631031,738.599106,218.181818,425.931446,1831.296572,1405.365127,5.616739,45.353701,0.16212,0.031257,2.870089,0.258469,0.218911,0.890052,0.009826,0.261353
1,1520.946551,1923.398824,681.139756,189.416554,223.066486,1752.510176,1529.443691,9.829388,129.835479,0.156291,0.038054,2.908602,0.247179,0.215644,0.749245,0.008963,0.234229
1,1571.712955,1891.555372,762.716378,186.950732,323.302264,1760.852197,1437.549933,8.066458,102.784822,0.123804,0.021955,2.740783,0.218292,0.206121,0.695577,0.011369,0.197232
1,1658.850753,2048.965036,652.509653,197.940798,222.136422,2179.92278,1957.786358,11.729366,193.645612,0.125327,0.035597,2.712013,0.211195,0.206266,0.657592,0.010872,0.183896
1,1718.951314,1927.182784,805.600515,213.100369,335.718882,2579.98089,2244.262007,8.485207,111.422507,0.148562,0.051651,2.904193,0.242128,0.211111,0.823674,0.009245,0.238346


In [8]:
df_male['label'] = 'male'
df_female['label'] = 'female'

In [9]:
df = pd.concat([df_male, df_female], ignore_index=True)
df

,mean,sd,median,mode,q25,q75,iqr,skew,kurt,zcr,energy,energy_entropy,spectral_centroid,spectral_spread,spectral_entropy,spectral_flux,spectral_rolloff,label
0,1808.092287,1990.436004,782.362729,98.835275,365.723794,3184.69218,2818.968386,5.096127,36.714087,0.132613,0.014204,2.776143,0.22061,0.207588,0.813328,0.007252,0.211261,male
1,2200.046049,2274.187035,955.15929,117.462955,351.461525,4268.851065,3917.38954,7.525938,85.611556,0.151185,0.031811,2.755876,0.257105,0.227426,0.944803,0.008513,0.247871,male
2,1675.355751,2147.506127,556.424261,119.998089,225.474117,2382.637219,2157.163102,8.714798,115.775205,0.112272,0.014426,2.627035,0.228569,0.225899,0.689215,0.015155,0.216506,male
3,1974.211667,1986.024539,925.484438,144.355789,451.305037,3254.032959,2802.727923,4.360345,26.907695,0.182504,0.015273,2.742456,0.267155,0.213,1.016115,0.008579,0.297207,male
4,1798.853498,1842.831468,949.25,135.25,376.75,2776.0,2399.25,6.768834,69.747551,0.089617,0.016476,2.795043,0.201541,0.208583,0.721911,0.012701,0.155377,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9061,2176.504278,2219.570297,1114.115308,182.902584,239.363817,3847.713718,3608.349901,13.680722,253.117683,0.116889,0.056615,2.891328,0.21092,0.210231,0.724152,0.010892,0.180177,female
9062,2027.550253,2212.227248,867.364747,173.82199,483.420593,2841.535777,2358.115183,8.323516,127.835342,0.204503,0.028066,2.814642,0.292336,0.220678,0.858237,0.010405,0.301394,female
9063,2414.310759,2515.698997,948.470209,195.169082,350.402576,5023.188406,4672.785829,12.584704,216.607914,0.22056,0.031528,2.833599,0.313691,0.225406,0.817461,0.01135,0.316728,female
9064,1615.2144,1751.907921,903.0123,197.44423,397.12127,1999.641155,1602.519886,9.76803,153.560083,0.133332,0.034586,2.79991,0.230782,0.209851,0.788931,0.009578,0.205444,female


In [10]:
df.to_csv('..\\dados\\data.csv', index=False)

In [4]:
teste = specprop('..\\temp\\audio.wav')
teste

c:\users\thiago.osorio\anaconda3\lib\pyaudioanalysis\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 901164 bytes, expected 901172 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


,mean,sd,median,mode,q25,q75,iqr,skew,kurt,zcr,energy,energy_entropy,spectral_centroid,spectral_spread,spectral_entropy,spectral_flux,spectral_rolloff
1,5109.564813,6218.487153,1141.832386,157.244318,532.670455,10094.957386,9562.286932,11.100156,181.34003,0.070256,0.011617,3.042737,0.176716,0.22051,0.378935,0.006634,0.109046
